In [1]:
import numpy as np
import pandas as pd
import sys
import re
from functions import *
from mining_functions import *
from collections import Counter
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
gitpath = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
df = pd.read_csv(get_path(path = gitpath) + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized.txt')
df_scores["student"] = df_scores["student"].astype('category')

## First we get the raw count of unique students using a sequence per time bin

In [5]:
### PARAMETERS
students = get_students()
attribute, level1, level2 = 'scaffolding','scaff','not'
B = 5  #number of bins
shortest_seq_length = 1
longest_seq_length = 10
CUT_OFF_SEQ_USE = 0.25 #we keep only sequences used once by at least 0.25% of students
N = int(CUT_OFF_SEQ_USE*len(students))

sequence_counts = get_sequence_use_by_timebin(df,students,B,attribute,level1,level2,shortest_seq_length,longest_seq_length,N)
for k,v in sequence_counts.iteritems():
    print k,v[0],v[1]

Getting sequence use over 5 time bins for 96 students split by scaffolding. 
            Keeping only sequences used once by at least 24 students.
CPI [ 12.   2.   1.   2.   6.] [ 18.   6.   1.   5.   4.]
CTcCP [ 4.  2.  3.  3.  2.] [ 3.  2.  7.  4.  1.]
TbP [ 22.   5.   4.   3.   3.] [ 16.   4.   4.   6.   5.]
CPC [ 25.  10.   9.  12.  21.] [ 28.  20.  17.  22.  26.]
TcCTc [  6.   9.  11.   5.  11.] [  7.  16.  14.  12.  13.]
TcCTcP [ 2.  5.  5.  2.  4.] [  3.  13.   7.   7.   6.]
TbI [ 14.   2.   4.   1.   0.] [ 12.   1.   0.   0.   7.]
TbC [ 29.   4.  10.   5.   5.] [ 20.   5.   4.   1.   7.]
TcCTcC [ 5.  3.  6.  1.  7.] [ 4.  5.  8.  8.  8.]
PC [ 37.  33.  30.  29.  32.] [ 45.  38.  35.  38.  38.]
CTsC [  8.  11.   8.   7.   2.] [ 5.  5.  7.  5.  5.]
TsC [  9.  19.  12.  10.   4.] [  8.   9.  12.   8.   7.]
PICI [ 3.  6.  3.  4.  3.] [ 3.  3.  0.  1.  2.]
CTsP [ 11.  13.  14.   8.   1.] [ 8.  9.  9.  8.  0.]
TbCTb [ 18.   2.   4.   3.   1.] [ 9.  2.  1.  0.  1.]
TsP [ 15.  22.  19.

### We can calculate the information gain of each sequence

In [22]:
def rank_sequences(sequence_counts):
    ranks = []
    for seq,counts in sequence_counts.iteritems():
        ranks.append((seq,calc_entropy(counts)))
    return sorted(ranks, key=lambda tup: tup[1])

In [23]:
rank_sequences(sequence_counts)

[('TbCTb', 2.4844175534200441),
 ('TbI', 2.585956877423087),
 ('TsPTsP', 2.8092643658278416),
 ('CTbP', 2.8333551742764422),
 ('CPI', 2.8395845044109702),
 ('TbC', 2.8524621935867494),
 ('TbP', 2.9404808965212519),
 ('TsPTsC', 3.028759373524017),
 ('CTsP', 3.0389659812492438),
 ('CPCI', 3.0898672472566897),
 ('ITcPC', 3.1022938734632239),
 ('TcCTcP', 3.1280639571270923),
 ('PICI', 3.1413806928025738),
 ('TsP', 3.1517243022938821),
 ('PCTb', 3.1704835645292526),
 ('TsCP', 3.1733269264042789),
 ('CTcCP', 3.1965547314219513),
 ('PCPCI', 3.2043743418986668),
 ('TcCP', 3.2317610168108519),
 ('IPC', 3.2379501831306046),
 ('TcCI', 3.2380834017526539),
 ('CPC', 3.243605250085277),
 ('TcCTcC', 3.2438561649084949),
 ('TsC', 3.2475500932914709),
 ('TcCTc', 3.2522063868222983),
 ('CPCP', 3.2533685946610689),
 ('CTsC', 3.25806187329577),
 ('PCTc', 3.2586521925671148),
 ('PCP', 3.2749048804069929),
 ('CPTc', 3.3049302290612359),
 ('PC', 3.3108329261983442),
 ('P', 3.3200302452161226),
 ('CPTs', nan)